<a href="https://colab.research.google.com/github/MeenTers/AIprototype/blob/main/Test_Acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [3]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance
import pandas as pd

In [4]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

# VGG model

In [5]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [6]:
vgg = VGG()

# Create Features Extraction Function

In [7]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

# Create Loading the data Function

In [11]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return loader

In [9]:
def get_euclidean(base_car,diff_view):
    A = []
    for i in range(len(diff_view)):
        x = distance.euclidean(base_car[i],diff_view[i])
        A.append(x)
    return(A)

# Test

In [12]:
car1 = get_dataset(r'/content/drive/MyDrive/test_acc/base/base')
dmg1 = get_dataset(r'/content/drive/MyDrive/test_acc/dmg/dmg/dmg')

In [13]:
id1b = get_features(vgg,car1)
id1v = get_features(vgg,dmg1)

100%|██████████| 1/1 [00:02<00:00,  2.30s/it]


In [14]:
dmg = get_euclidean(id1b,id1v)
dmg 

[11.785266876220703, 26.288467407226562, 4.0519914627075195, 15.6392822265625]

In [18]:
car2 = get_dataset(r'/content/drive/MyDrive/test_acc/base/base')
dmg2 = get_dataset(r'/content/drive/MyDrive/test_acc/dmg/ldmg/ldmg')

In [19]:
id2b = get_features(vgg,car2)
id2v = get_features(vgg,dmg2)

100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


In [20]:
ldmg = get_euclidean(id2b,id2v)
ldmg 

[1.6285629272460938,
 21.440866470336914,
 3.2205731868743896,
 14.069794654846191]

In [21]:
car3 = get_dataset(r'/content/drive/MyDrive/test_acc/base/base')
dmg3 = get_dataset(r'/content/drive/MyDrive/test_acc/dmg/mdmg/mdmg')

In [22]:
id3b = get_features(vgg,car3)
id3v = get_features(vgg,dmg3)

100%|██████████| 1/1 [00:02<00:00,  2.31s/it]


In [23]:
mdmg = get_euclidean(id3b,id3v)
mdmg 

[26.43695831298828, 35.972679138183594, 38.565250396728516, 27.064756393432617]

In [24]:
car4 = get_dataset(r'/content/drive/MyDrive/test_acc/base/base')
dmg4 = get_dataset(r'/content/drive/MyDrive/test_acc/base/base')

In [25]:
id4b = get_features(vgg,car4)
id4v = get_features(vgg,dmg4)

100%|██████████| 1/1 [00:00<00:00,  4.43it/s]


In [26]:
nodmg = get_euclidean(id4b,id4v)
nodmg 

[0.0, 0.0, 0.0, 0.0]

In [28]:
def get_dmg(eucli):
  for j in range(4):
          if round(eucli[j]) < 0.5:
              print('N')
          elif   0.5 <= round(eucli[j]) <= 10.5:
              print('L')
          elif 10.5 < round(eucli[j]) <= 30:
              print('M')
          elif round(eucli[j]) > 30:
               print('H')

In [30]:
get_dmg(nodmg)

N
N
N
N


In [31]:
get_dmg(ldmg)

L
M
L
M


In [32]:
get_dmg(dmg)

M
M
L
M


In [33]:
get_dmg(mdmg)

M
H
H
M


In [41]:
from sklearn.metrics import confusion_matrix

# Predicted values
y_pred = ["N", "N", "N", "N", "L","M","L","M","M","M","L","M","M","H","H","M"]
# Actual values
y_act = ["N", "N", "N", "N", "L","L","L","L","M","M","M","M","H","H","H","H"]

In [70]:
# Predicted values
y_pred = ["N", "N", "N", "N", "L","M","L","M","M","M","L","M","M","H","H","M"]
# Actual values
y_act = ["N", "N", "N", "N", "L","L","L","L","M","M","M","M","H","H","H","H"]

In [77]:
from torchvision.transforms.functional import normalize
confusion_matrix(y_act, y_pred,normalize='true')

array([[0.5 , 0.  , 0.5 , 0.  ],
       [0.  , 0.5 , 0.5 , 0.  ],
       [0.  , 0.25, 0.75, 0.  ],
       [0.  , 0.  , 0.  , 1.  ]])

In [67]:
class_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')